In [52]:
import pandas as pd 
from transformers import pipeline
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import DistilBertForSequenceClassification
from transformers import Trainer , TrainingArguments
import torch
from huggingface_hub import notebook_login

In [53]:
notebook_login()

In [54]:
classifier = pipeline("text-classification", model = "palakagl/bert_TextClassification")

Let's use the data from sanofi 

In [55]:
df = pd.read_excel("C:/Users/edenl/Desktop/file_rouge/notebook/original_data/data_sa.xlsx" )
df


,Title,Sentiment,Text content,Item Type,Path
0,1963750159122432,neutral,Is there a concise and Sanofi-specific explana...,Item,sites/CollaborationApplicationTeam/Lists/senti...
1,1964814211776512,neutral,"When you open ""See all news<https://sanofi.sha...",Item,sites/CollaborationApplicationTeam/Lists/senti...
2,1965439534342144,positive,It's a great day for us!! Today we are live wi...,Item,sites/CollaborationApplicationTeam/Lists/senti...
3,1966084325793792,positive,"GLOCAL = Is this what I am thinking, a close c...",Item,sites/CollaborationApplicationTeam/Lists/senti...
4,1966159410651136,positive,Good luck! Congrats with your new position!,Item,sites/CollaborationApplicationTeam/Lists/senti...
...,...,...,...,...,...
166,1994441334685696,positive,Congrats and good luck on your new challenge!,Item,sites/CollaborationApplicationTeam/Lists/senti...
167,1994649610772480,positive,"Rush - La Villa Strangiato\nI love me a great,...",Item,sites/CollaborationApplicationTeam/Lists/senti...
168,1995359947137024,positive,💎 #worldqualityweek 💎\nplease see this great t...,Item,sites/CollaborationApplicationTeam/Lists/senti...
169,1995402667737088,neutral,Michael Bublé - Home [Official Music Video] - ...,Item,sites/CollaborationApplicationTeam/Lists/senti...


In [56]:
df_head = df.head(100)


In [57]:
samples = df["Text content"][1:100]
samples

1     When you open "See all news<https://sanofi.sha...
2     It's a great day for us!! Today we are live wi...
3     GLOCAL = Is this what I am thinking, a close c...
4           Good luck! Congrats with your new position!
5                                      Congrats Marcel!
                            ...                        
95                                        Bravo Cécile!
96    Cool!! We are using Scrumban in AR for Sarclis...
97    Hi Schaefer, Ute /DE, just a quick update to l...
98    Thanks Khalid! Fantastic session and lots of l...
99    what a wonderful & promising start, an eye-ope...
Name: Text content, Length: 99, dtype: object

In [58]:

sequence = samples.astype(str).values.tolist()
#print(sequence)


In [59]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokens = tokenizer.tokenize(sequence , truncation=True, padding=True , is_split_into_words=True)
tokens

['When',
 'you',
 'open',
 '"',
 'See',
 'all',
 'news',
 '<',
 'https',
 ':',
 '/',
 '/',
 'sa',
 '##no',
 '##fi',
 '.',
 'share',
 '##point',
 '.',
 'com',
 '/',
 'sites',
 '/',
 'Buzz',
 '/',
 'Site',
 '##P',
 '##ages',
 '/',
 'search',
 '.',
 'as',
 '##p',
 '##x',
 '?',
 'q',
 '=',
 '*',
 '&',
 'vertical',
 '=',
 'Buzz',
 '%',
 '20',
 '##N',
 '##ews',
 '>',
 '"',
 'on',
 'the',
 'Buzz',
 'Home',
 '##page',
 ',',
 'you',
 'have',
 'to',
 'wait',
 'quite',
 'some',
 'time',
 'before',
 'you',
 'see',
 'any',
 'news',
 '.',
 'There',
 'are',
 'currently',
 '39',
 '##4',
 'news',
 'articles',
 'to',
 'be',
 'found',
 '–',
 'once',
 'they',
 'are',
 'displayed',
 '.',
 'Any',
 'plans',
 'to',
 'speed',
 'this',
 '?',
 'It',
 "'",
 's',
 'a',
 'great',
 'day',
 'for',
 'us',
 '!',
 '!',
 'Today',
 'we',
 'are',
 'live',
 'with',
 'the',
 'MVP',
 'of',
 'the',
 'Du',
 '##pi',
 'A',
 '##ccelerator',
 'Spain',
 ':',
 ')',
 'Con',
 '##gra',
 '##ts',
 'to',
 'all',
 'the',
 'trans',
 '##vers'

In [60]:
training_args = TrainingArguments(
    output_dir="notebook", 
    push_to_hub=True,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16, 
    warmup_steps=500,
    learning_rate=5e-5, 
    weight_decay=0.01,
    logging_dir="notebook", 
    logging_steps=10,


)

model = DistilBertForSequenceClassification.from_pretrained(classifier)
trainer = Trainer(
    model=model,
    args=training_args, 
    train_dataset=tokens,
    eval_dataset=tokens

)

trainer.train()


HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/api/models/%3Ctransformers.pipelines.text_classification.TextClassificationPipeline%20object%20at%200x000001C80E1F0700%3E